In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://ptnacamara.org.br/portal/category/portal/noticias/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h2', class_='entry-title')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://ptnacamara.org.br/portal/2018/10/05/cut-denuncia-violacoes-de-direitos-humanos-em-audiencia-publica-nos-eua/
https://ptnacamara.org.br/portal/wp-content/themes/woohoo/images/favicon.png
100% [..................................................................................] 401 / 401favicon.png
C:\Users\admin\rss3\src\midia_postagem\images\favicon.png
['CUT denuncia violações de Direitos Humanos em audiência pública nos EUA']
news_in_db: False
 -- no categories -- 
https://ptnacamara.org.br/portal/2018/10/05/constituicao-cidada-completa-30-anos-nesta-sexta-feira/
https://ptnacamara.org.br/portal/wp-content/themes/woohoo/images/favicon.png
100% [..................................................................................] 401 / 401favicon.png
C:\Users\admin\rss3\src\midia_postagem\images\favicon.png
['Constituição Cidadã completa 30 anos nesta sexta-feira']
news_in_db: False
 -- no categories -- 
https://ptnacamara.org.br/portal/2018/10/05/ministro-do-tst-afirma-que-refor

In [6]:
noticias[0]

<h2 class="entry-title"><a href="https://ptnacamara.org.br/portal/2018/10/05/cut-denuncia-violacoes-de-direitos-humanos-em-audiencia-publica-nos-eua/"><span>CUT denuncia violações de Direitos Humanos em audiência pública nos EUA</span></a></h2>